In [5]:
#numpy and pandas for data manipulation
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold

from sklearn.model_selection import StratifiedKFold

from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import cross_val_predict

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split
from sklearn import metrics

#sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

from itertools import product


samples = pd.read_csv('data/application_train.csv')

# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in samples:
    if samples[col].dtype == 'bool':
        # If 2 or fewer unique categories
        if len(list(samples[col].unique())) <= 2:
            # Train on the training data
            le.fit(samples[col])
            # Transform 
            samples[col] = le.transform(samples[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
print('%d columns were label encoded.' % le_count)

train_label = samples['TARGET']
samples = samples.drop(columns=['TARGET'])

# one-hot encoding of categorical variables
samples = pd.get_dummies(samples)

# Median imputation of missing values
imputer = SimpleImputer(strategy = 'median')

# Scale each feature to -1 and 1
scaler = StandardScaler()

# Fit on the training data
imputer.fit(samples)

# Transform both training and testing data
samples = imputer.transform(samples)

# Repeat with the scaler
scaler.fit(samples)
samples = scaler.transform(samples)

print('Training data shape: ', samples.shape)

0 columns were label encoded.
Training data shape:  (307511, 245)


In [6]:
x_train,x_test, y_train, y_test = train_test_split(samples, train_label, test_size=0.2)

In [7]:
def classify(arr, threshold):
    return [1 if row[1] >= threshold else 0 for row in arr]

In [9]:
# C, max iter, threshold, weight
comb_hyperparameters = product(tuple((1,0.1,0.01,0.001,0.0001)), tuple((100,200,400, 700)), 
                               tuple((None, {0:0.1238,1:0.8762},{0:0.2238,1:0.7762}, 'balanced')),
                              tuple((0.2, 0.4, 0.5)) )

# for hp in comb_hyperparameter:
# 80-20 train-test case
kf = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)
total_hidden_nodes = 20
mlp = MLPRegressor(hidden_layer_sizes=(total_hidden_nodes,),activation='relu', solver='adam', 
                    batch_size=200, learning_rate_init=0.001, max_iter=100, alpha=0.0001, shuffle=True)
jpt = cross_val_predict(estimator=mlp, X=x_train, y=y_train, cv=kf)
# y_pred = classify(cross_val_predict(estimator=mlp, X=x_train, y=y_train, cv=kf), hp[3])

print("Hyperparameters - {}  roc - {}".format(hp,metrics.roc_auc_score(y_train, y_pred)))

NameError: name 'hp' is not defined